We develop validations for circuit composition.

In [1]:
import os
os.environ["LOGLEVEL"] = "PROD"

import numpy as np
import pandas as pd
from dmt.tk.phenomenon import Phenomenon
from neuro_dmt.analysis.circuit import BrainCircuitAnalysis
from neuro_dmt.models.bluebrain.circuit.model import BlueBrainCircuitModel
from neuro_dmt.models.bluebrain.circuit.adapter import BlueBrainCircuitAdapter
from neuro_dmt.models.bluebrain.circuit.mock.test.mock_circuit_light\
    import circuit_composition, circuit_connectivity
from neuro_dmt.models.bluebrain.circuit.mock.circuit import\
    MockCircuit

('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')


In [2]:
adapter = BlueBrainCircuitAdapter()
adapter.get_method_description("cell_density")

'\n                Cells were counted in a box with sides of length [50. 50. 50.] um.\n                Each cube was centered at a cell that was randomly sampled from \n                a population described by a cell-query.\n                '

In [3]:
mock_circuit =\
    MockCircuit.build(
        circuit_composition,
        circuit_connectivity)
circuit_model =\
    BlueBrainCircuitModel(
        mock_circuit,
        label="BlueBrainCircuitModelMock")
adapter =\
    BlueBrainCircuitAdapter()

In [5]:
from dmt.tk.parameters import Parameters
from dmt.tk.plotting import Bars
from dmt.tk.plotting.multi import MultiPlot
from neuro_dmt.analysis.circuit.composition.interfaces import CellDensityAdapterInterface
from neuro_dmt.data import rat

layers =[
    "L{}".format(layer) for layer in range(1, 7)
]
regions =[
    "S1HL", "S1FL", "S1Sh", "S1Tr"
]

def _append_region(reference_dataframe):
    return pd.concat([
        reference_dataframe\
            .reset_index()\
            .assign(region=region)\
            .set_index(["region", "layer"])
        for region in regions])
    
reference_data =\
    dict(
        DeFelipe2017=_append_region(rat.defelipe2017.summary_measurement.samples(1000)),
        DeFelipe2014=_append_region(rat.defelipe2014.summary_measurement.samples(1000)),
        meyer2010=_append_region(rat.meyer2010.samples(1000)))
analysis = BrainCircuitAnalysis(
    phenomenon=Phenomenon(
        "Cell Density",
        "Number of cells in a unit volumne",
        group="Composition"),
        AdapterInterface=CellDensityAdapterInterface,
        reference_data=reference_data,
        measurement_parameters=Parameters(
            pd.DataFrame({
                "region": [r for r in regions for _ in layers],
                "layer": [l for _ in regions for l in layers]})),
        plotter=Bars(
            xvar="layer",
            xlabel="Layer",
            yvar="cell_density",
            ylabel="Cell Density",
            gvar="dataset"))

In [ ]:
analysis.AdapterInterface.__measurement__

In [ ]:
cell_density_measurement =\
    analysis.get_measurement(
        circuit_model,
        adapter)

In [ ]:
cell_density_measurement.head()

In [ ]:
(adapter.get_cell_density(circuit_model, layer="L6"),
 adapter.get_cell_density(circuit_model, layer=6))

In [ ]:
data =\
    analysis._with_reference_data(
        cell_density_measurement)

In [ ]:
list(data.values())[0].columns[0]

In [ ]:
from dmt.data.observation import measurement
multiplotter =\
    MultiPlot(
        mvar="region",
        plotter=Bars(
            xvar="layer",
            xlabel="Layer",
            yvar="cell_density",
            ylabel="Cell Density",
            gvar="dataset"))
multiplotter.get_figures(data)

In [14]:
from pathlib import Path
from neuro_dmt.analysis.reporting import CheetahReporter

#report =\
#    analysis(circuit_model, adapter)
reporter =\
    CheetahReporter(path_output_folder=Path.cwd().joinpath("validations"))

reporter.post(report)

'/Users/vishalsoodmuchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/test/validations/cell_density/20191119/211708'

In [12]:
reporter._save_figures(report, output_folder="validations")

('validations/figures', {'figure': 'validations/figures/figure.png'})

In [ ]:
from neuro_dmt.models.bluebrain.circuit.test import BlueBrainCircuitAnalysisTest

phenomenon = Phenomenon(
    "Cell Density",
    "Number of cells in a unit volume.",
    group="Composition"
)
layers =[
    "L{}".format(layer) for layer in range(1, 7)
]
regions =[
    "S1HL", "S1FL", "S1Sh", "S1Tr"
]
def _append_region(reference_dataframe):
    return pd.concat([
        reference_dataframe\
            .reset_index()\
            .assign(region=region)\
            .set_index(["region", "layer"])
        for region in regions])
    
reference_data =\
    dict(
        DeFelipe2017=_append_region(rat.defelipe2017.summary_measurement.samples(1000)),
        DeFelipe2014=_append_region(rat.defelipe2014.summary_measurement.samples(1000)),
        meyer2010=_append_region(rat.meyer2010.samples(1000)))
analysis_test = BlueBrainCircuitAnalysisTest(
    analysis=BrainCircuitAnalysis(
        phenomenon=phenomenon,
        AdapterInterface=CellDensityAdapterInterface,
        reference_data=reference_data,
        measurement_parameters=Parameters(
            pd.DataFrame({
                "region": [
                    region
                    for _ in layers
                    for region in regions],
                "layer": [
                    layer
                    for layer in layers
                    for _ in regions]})),
    plotter=MultiPlot(
        mvar="region",
        plotter=Bars(
            xvar="layer",
            xlabel="Layer",
            yvar=phenomenon.label,
            ylabel=phenomenon.name,
            gvar="dataset"))))


In [ ]:
data = analysis_test.test_get_measurement(circuit_model, sample_size=2)

In [ ]:
analysis.plotter.get_figures(data)

In [ ]:
from collections import OrderedDict
import pandas
import seaborn
from dmt.tk.field import Field, lazyfield, WithFields

class MultiPlot(WithFields):
    """
    Using a plotter-type instance, create several plots, that will be
    returned in a dictionary.
    """
    plotter = Field(
        """
        Plotter type instance.
        """)
    mvar = Field(
        """
        Multi-plot variable.
        Name of the column in the dataframes to be plotted that will be varied
        to subset the data, feed it to `self.plotter` to get individual plots
        for the values of `mvar`.
        """)
    
    def _get_sub_figure(self,
            value_mvar,
            dataframe_subset,
            caption,
            *args,
            **kwargs):
        """..."""
        return self.plotter.get_figure(
            dataframe_subset,
            *args,
            caption=caption,
            title="{} {}".format(self.mvar, value_mvar),
            **kwargs
        )

    def get_figures(self,
            data,
            *args,
            caption="Caption not provided",
            **kwargs):
        """
        Returns
        ================
        OrderedDict mapping value of `self.mvar` to its plot.
        """


        dataframe = self.plotter.get_dataframe(data)
        values_mvar = dataframe[self.mvar].unique()
        return OrderedDict([
            (value_mvar,
             self._get_sub_figure(
                 value_mvar,
                 dataframe[dataframe[self.mvar] == value_mvar],
                 caption,
                *args, **kwargs))
             for value_mvar in values_mvar])

In [ ]:
plotter =\
    MultiPlot(
        mvar="region",
        plotter=Bars(
            xvar="layer",
            xlabel="Layer",
            yvar=phenomenon.label,
            ylabel=phenomenon.name,
            gvar="dataset"))
dataframe = plotter.plotter.get_dataframe(data)

In [ ]:
plotter._get_sub_figure("S1HL", dataframe[dataframe[plotter.mvar] == "S1HL"], caption="Not available")

In [ ]:
dataframe[dataframe[plotter.mvar] == "S1HL"]

In [ ]:
dataframe.tail()

In [ ]:
figures = analysis.plotter.get_figures(data)

In [ ]:
from dmt.data.observation import measurement
summary = measurement.concat_as_summaries(data)

In [ ]:
len(layers) * len(regions) * (1. + len(reference_data))

In [ ]:
from dmt.tk.utils.string_utils import make_name, make_text_like
print(make_name("S1HL", keep_original_capitalization=True))
print(make_text_like("S1HL", separator="_", with_capitalized_words=True, keep_original_capitalization=True))
print(make_name('_'.join(("pre_synaptic", "mtype")), separator='_'))

In [ ]:
def make_text_like(
        string,
        separator,
        with_capitalized_words,
        with_characters_to_remove=[
            ',', ':', '&', '#', '/',
            '\\', '$', '?', '^', ';', '.'],
        keep_original_capitalization=False):
    """
    Turn a string into normal text like.
    Keep only ASCII characters, removing the provided list.
    And capitalize...
    """
    if keep_original_capitalization:
        with_capitalized_words = False
    def __removed(word):
        return ''.join(c for c in word if c not in with_characters_to_remove)

    def __captialzed(word):
        return word.capitalize() if with_capitalized_words else word

    return separator\
        .join(
            __captialzed(
                __removed(
                    word if keep_original_capitalization else word.lower()))
            for word in string.strip().split(' ')
            if len(word) > 0)

In [ ]:
make_text_like("S1HL", separator="_", with_capitalized_words=True, keep_original_capitalization=True)

In [ ]:
"S1HL".capitalize()

In [ ]:
from dmt.tk.phenomenon import Phenomenon
Phenomenon(
    "Cell Density",
    "...."
).name

In [ ]:
make_name("cell_density", separator='_')

In [ ]:
import pandas as pd
from dmt.tk.plotting import LinePlot

post_synaptic_mtypes = ["L23_MC", "L6_TPC:A", "L4_UPC", "L5_TPC:A"]
pre_synaptic_mtypes = ["L23_MC", "L6_TPC:A", "L4_UPC", "L5_TPC:A"]
number_pathways = len(pre_synaptic_mtypes) * len(post_synaptic_mtypes)
some_data = pd.DataFrame({
    ("post_synaptic", "mtype"): [
        post for post in post_synaptic_mtypes for _ in pre_synaptic_mtypes for _ in range(10)],
    ("pre_synaptic", "mtype"): [
        pre for _ in post_synaptic_mtypes for pre in pre_synaptic_mtypes for _ in range(10)],
    "soma_distance": np.concatenate(
        [np.linspace(1., 1000., 10) for _ in range(number_pathways)], axis=0),
    "afferent_connections": np.random.uniform(
        0, 20, size=number_pathways*10)
})

In [ ]:
plotter = LinePlot(
    xvar="soma_distance",
    xlabel="Soma Distnace",
    yvar="afferent_connections",
    ylabel="Afferent Connections",
    gvar=("pre_synaptic", "mtype"),
    fvar=("post_synaptic", "mtype"),
    number_columns=2,
    drawstyle="steps-mid")

In [ ]:
plotter.get_figure(some_data)

In [ ]:
import seaborn
grid =\
    seaborn.FacetGrid(
        some_data,
        col=("post_synaptic", "mtype"),
        hue=("pre_synaptic", "mtype"),
        col_wrap=2,
        legend_out=True
)
grid.map(
    seaborn.lineplot,
    "soma_distance",
    "afferent_connections",
    drawstyle="steps-mid",
    alpha=0.7
).set_titles(
    "{col_name} pre synaptic"
).add_legend()

In [ ]:
from neuro_dmt.models.bluebrain.circuit.adapter import BlueBrainCircuitAdapter
adapter = BlueBrainCircuitAdapter()

In [ ]:
adapter.get_number_connections_afferent.__doc__